In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder 
from sklearn.metrics import mean_squared_log_error

# np.sqrt(mean_squared_log_error(actual, predicted))

import xgboost as xgb

In [ ]:
from google.colab import files
uploaded = files.upload()



#train_df =  pd.read_csv('C:\Users\nkdhe\OneDrive\Desktop\Machine Learning A-Z Template/ParticipantData_BTPC/Train.csv')
#test_df = pd.read_csv('C:\Users\nkdhe\OneDrive\Desktop\Machine Learning A-Z Template/ParticipantData_BTPC/Test.csv')

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
train_df= pd.read_csv('Train.csv')
test_df= pd.read_csv('Test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
len(pd.unique(train_df['session_id']))==len(train_df)

In [ ]:
train = train_df.copy()
test = test_df.copy()

# **BASIC EDA**

In [ ]:
train = train.drop(labels=['session_id','client_agent'],axis = 1)
test = test.drop(labels=['session_id','client_agent'],axis = 1)

In [ ]:
train['device_details'].value_counts()

In [ ]:
train.describe()

In [ ]:
train['session_number'].value_counts()

In [ ]:
train["date"] = pd.to_datetime(train['date'])
test["date"] = pd.to_datetime(test['date'])

train["year"] = train['date'].dt.year
test["year"] = test['date'].dt.year

train["month"] = train['date'].dt.month
test["month"] = test['date'].dt.month

train['day']= train['date'].dt.day
test["day"] = test['date'].dt.day

train['diff_date'] = ((train['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)
test['diff_date'] = ((test['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)

In [ ]:
train.head()

In [ ]:
l = pd.unique(train["device_details"])
device_encoding = {}
x = 0
for i in l:
    if i not in device_encoding:
        device_encoding[i] = x
        x+=1
print(device_encoding)

train["device_encoding"] = train["device_details"].map(device_encoding)
test["device_encoding"] = test["device_details"].map(device_encoding)

In [ ]:
train.head()

In [ ]:
train= train[pd.notnull(train["time_spent"])]

# **MODELLING**

In [ ]:
cols = ['session_number','device_encoding','purchased','added_in_cart','checked_out','year','month','day','diff_date']
#cols = ['session_number','purchased','added_in_cart','checked_out']
X = train[cols]
Y = train["time_spent"]

In [ ]:
x_tr,x_tst,y_tr,y_tst = train_test_split(X,Y,test_size=0.20,random_state=0)


# **FILE CREATION**

In [ ]:
sub = pd.read_csv('../input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Sample Submission.csv')
sub["time_spent"]=abs(ans)

In [ ]:
sub.to_csv('sub1_xgb.csv', index=False)

In [ ]:
# Neural Network

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

### Load the data

# First we load the entire CSV file into an m x 3


# We extract all rows and the first 2 columns, and then transpose it
x_dataset = x_tr

# We extract all rows and the last column, and transpose it
y_dataset = y_tr

# And make a convenient variable to remember the number of input columns
n = 4


### Model definition ###

# First we define the trainable parameters A and b 
A = torch.randn((1, n), requires_grad=True)
b = torch.randn(1, requires_grad=True)

# Then we define the prediction model
def model(x_input):
    return A.mm(x_input.float()) + b


### Loss function definition ###

def loss(y_predicted, y_target):
    return ((y_predicted - y_target)**2).sum()

### Training the model ###

# Setup the optimizer object, so it optimizes a and b.
optimizer = optim.Adam([A, b], lr=0.1)

# Main optimization loop
for t in range(2000):
    # Set the gradients to 0.
    optimizer.zero_grad()
    # Compute the current predicted y's from x_dataset
    y_predicted = model(x_dataset)
    # See how far off the prediction is
    current_loss = loss(y_predicted, y_dataset)
    # Compute the gradient of the loss with respect to A and b.
    current_loss.backward()
    # Update A and b accordingly.
    optimizer.step()
    print(f"t = {t}, loss = {current_loss}, A = {A.detach().numpy()}, b = {b.item()}")

In [ ]:
# 2 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_val=x_tr.iloc[0:400, :]                                                        # Validation dataset
y_val=y_tr.iloc[0:400]

y_tr, y_tst, y_val = y_tr.astype(float), y_tst.astype(float), y_val.astype(float)
#x_tr, x_tst, x_val = x_tr.astype(float), x_tst.astype(float), x_val.astype(float)

# scaling 
#scaler = MinMaxScaler()
#x_tr = scaler.fit_transform(x_tr)
#x_val = scaler.transform(x_val)
#x_tst = scaler.transform(x_tst)
x_tr, y_tr = np.array(x_tr), np.array(y_tr)
x_val, y_val = np.array(x_val), np.array(y_val)
x_tst, y_tst = np.array(x_tst), np.array(y_tst)


In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


class RegressionDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
train_dataset = RegressionDataset(torch.from_numpy(x_tr).float(), torch.from_numpy(y_tr).float())
val_dataset = RegressionDataset(torch.from_numpy(x_val).float(), torch.from_numpy(y_val).float())
test_dataset = RegressionDataset(torch.from_numpy(x_tst).float(), torch.from_numpy(y_tst).float())

EPOCHS = 400
BATCH_SIZE = 64
LEARNING_RATE = 0.1
NUM_FEATURES = len(X.columns)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class MultipleRegression(nn.Module):
    def __init__(self, num_features):
        super(MultipleRegression, self).__init__()
        self.layer_1 = nn.Linear(num_features, 16)
        self.layer_2 = nn.Linear(16, 32)
        self.layer_3 = nn.Linear(32, 96)
        self.layer_4 = nn.Linear(96, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_6 = nn.Linear(32, 16)
        self.layer_out = nn.Linear(16,1)
        self.relu = nn.Tanh()
    def forward(self, inputs):
      x = self.relu(self.layer_1(inputs))
      x = self.relu(self.layer_2(x))
      x = self.relu(self.layer_3(x))
      x = self.relu(self.layer_4(x))
      x = self.relu(self.layer_5(x))
      x = self.relu(self.layer_6(x))
      x = self.layer_out(x)
      return (x)
    def predict(self, test_inputs):
      x = self.relu(self.layer_1(test_inputs))
      x = self.relu(self.layer_2(x))
      x = self.relu(self.layer_3(x))
      x = self.relu(self.layer_4(x))
      x = self.relu(self.layer_5(x))
      x = self.relu(self.layer_6(x))
      x = self.layer_out(x)
      return (x)



model = MultipleRegression(NUM_FEATURES)
model.to(device)
print(model)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


loss_stats = {
    'train': [],
    "val": []
}

print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
model.train()
for X_train_batch, y_train_batch in train_loader:

  X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
  optimizer.zero_grad()
        
  y_train_pred = model(X_train_batch)
        
  train_loss = criterion(y_train_pred, y_train_batch.unsqueeze(1))
        
  train_loss.backward()
  optimizer.step()
        
  train_epoch_loss += train_loss.item()
        
        
# VALIDATION
with torch.no_grad():
  
  val_epoch_loss = 0
  model.eval()
  for X_val_batch, y_val_batch in val_loader:
    X
    X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
    y_val_pred = model(X_val_batch)
                        
    val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))
            
    val_epoch_loss += val_loss.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}')    
   




In [ ]:
y_pred_list = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_pred_list.append(y_test_pred.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
score5=np.sqrt(mean_squared_log_error(y_tst,y_pred_list))


In [ ]:
score5

In [ ]:

test=np.array(test[cols])


test_dataset = RegressionDataset(torch.from_numpy(test).float(),torch.from_numpy(test).float())
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [ ]:
y_pred_list = []
with torch.no_grad():
    model.eval()
    for X_batch,_ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_pred_list.append(y_test_pred.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
sub = pd.read_csv('Sample Submission.csv')
sub["time_spent"]=y_pred_list

In [ ]:
sub.to_csv('sub2_NN4.csv', index=False)

In [ ]:
test